# Getting summary stats from the minION

In [4]:
# Process the datasets
import pandas as pd
import os


data_dir = '/Users/ariane/Documents/code/MinION/manucript/notebooks/Ape AGW/Data/'
# First we want to map between the dataset with the LC-MS results and the raw data columns
evseq_df = pd.read_csv(f'{data_dir}ep_EVSeqL/20240422-YL.csv', index_col=0)
evseq_df['Experiment'] = '20240422'
# Need to have an experiment tag as well
evseq_df['id'] = [f'{p[0]}_{p[1]}_{p[2]}' for p in evseq_df[['Experiment', 'Plate', 'Well']].values]

# First we want to map between the dataset with the LC-MS results and the raw data columns
evseq2_df = pd.read_csv(f'{data_dir}MinION parlq round2/yl-20240502.csv', index_col=0)
evseq2_df['Experiment'] = '20240502'
evseq2_df['id'] = [f'{p[0]}_{p[1]}_{p[2]}' for p in evseq2_df[['Experiment', 'Plate', 'Well']].values]
# 300-1.csv 
evseq_df = pd.concat([evseq2_df, evseq_df])

# Also get all the LC-MS data
files = os.listdir(f'{data_dir}lcms-processed/')
data_df = pd.DataFrame()
for f in files:
    df = pd.read_csv(f'{data_dir}lcms-processed/{f}')
    # Make the ID based on the filename
    plate_id = f.split('.csv')[0]
    df['Plate'] = plate_id
    df['Experiment'] = '20240422'
    # Now add in the actual ID too
    df['id'] = [f'{p[0]}_{p[1]}_{p[2]}' for p in df[['Experiment', 'Plate', 'Well']].values]
    data_df = pd.concat([data_df, df])

# Also get all the LC-MS data for the second run
files = os.listdir(f'{data_dir}MinION parlq round2/')
for f in files:
    if f != 'yl-20240502.csv' and '.csv' in f:
        df = pd.read_csv(f'{data_dir}MinION parlq round2/{f}')
        # Make the ID based on the filename
        plate_id = f.split('.csv')[0]
        df['Plate'] = plate_id
        df['Experiment'] = '20240502'
        # Now add in the actual ID too
        df['id'] = [f'{p[0]}_{p[1]}_{p[2]}' for p in df[['Experiment', 'Plate', 'Well']].values]
        data_df = pd.concat([data_df, df])

# Join the datasets
# Set the index for both of them 
data_df.set_index('id', inplace=True)
evseq_df.set_index('id', inplace=True)
df = data_df.join(evseq_df, how='inner', lsuffix='_data')

# Save to a csv
df.to_csv(f'{data_dir}output.csv')

# Get summary stats from the dataset

1. Make reference/comparison to the parent for each of the data

In [9]:
from sciutil import *


na_df = df[df['Variant'] == '#N.A.#']
parent_df = df[df['Variant'] == '#PARENT#']
variant_df = df[~df['Variant'].isin(['#PARENT#', '#N.A.#'])]
print(len(na_df), len(parent_df), len(variant_df))

42 106 236


In [10]:
df

,Well_data,Row,Column,Plate_data,cis,trans,Experiment_data,Variant_data,Normalized Activity,barcode_plate,...,Well,Variant,Alignment Count,Average mutation frequency,P value,P adj. value,Mutations,nc_variant,aa_variant,Experiment
id,,,,,,,,,,,,,,,,,,,,,
20240422_500-2_A1,A1,A,1,500-2,1.060808e+04,0.000000e+00,20240422,NaN,NaN,4,...,A1,#N.A.#,0.0,NaN,NaN,NaN,#N.A.#,ATGGCGGTTCCCGGCTACGATTTTGGCAAAGTCCCGGATGCCCCAA...,MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALK...,20240422
20240422_500-2_A2,A2,A,2,500-2,5.961494e+05,4.705581e+05,20240422,NaN,NaN,4,...,A2,#N.A.#,0.0,NaN,NaN,NaN,#N.A.#,ATGGCGGTTCCCGGCTACGATTTTGGCAAAGTCCCGGATGCCCCAA...,MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALK...,20240422
20240422_500-2_A3,A3,A,3,500-2,8.323569e+05,7.204202e+05,20240422,NaN,NaN,4,...,A3,T326G,22.0,0.863636,2.290000e-29,2.200000e-27,M109R,ATGGCGGTTCCCGGCTACGATTTTGGCAAAGTCCCGGATGCCCCAA...,MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALK...,20240422
20240422_500-2_A4,A4,A,4,500-2,5.993150e+05,4.741821e+05,20240422,NaN,NaN,4,...,A4,#PARENT#,12.0,NaN,NaN,NaN,#PARENT#,ATGGCGGTTCCCGGCTACGATTTTGGCAAAGTCCCGGATGCCCCAA...,MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALK...,20240422
20240422_500-2_A5,A5,A,5,500-2,1.650538e+05,1.827627e+05,20240422,NaN,NaN,4,...,A5,T21C_G25DEL_A179T_A240T_A320G,21.0,0.980952,1.180000e-140,1.130000e-138,-,Deletion,Deletion,20240422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240422_300-1_H8,H8,H,8,300-1,0.000000e+00,0.000000e+00,20240422,Sterile,0.0000,1,...,H8,NaN,7.0,NaN,NaN,NaN,#PARENT#,ATGGCGGTTCCCGGCTACGATTTTGGCAAAGTCCCGGATGCCCCAA...,MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALK...,20240422
20240422_300-1_H9,H9,H,9,300-1,8.062858e+04,8.377119e+04,20240422,Variant,-3142.6048,1,...,H9,T317A_A562G,66.0,0.915385,3.200000e-177,3.070000e-175,L106*_T188A,ATGGCGGTTCCCGGCTACGATTTTGGCAAAGTCCCGGATGCCCCAA...,MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALK...,20240422
20240422_300-1_H10,H10,H,10,300-1,2.125053e+06,1.702450e+06,20240422,Variant,422602.6030,1,...,H10,T21C_T289C_T326C,52.0,0.532051,3.830000e-82,3.670000e-80,M109T,ATGGCGGTTCCCGGCTACGACTTTGGCAAAGTCCCGGATGCCCCAA...,MAVPGYDFGKVPDAPISDADFESLKKTVMWGEEDEKYRKMACEALK...,20240422
